<a href="https://colab.research.google.com/github/WafaSanaa/Computer_Vision/blob/main/Classification_(Cancer_de_poumons).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas

In [2]:
pip install numpy

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
pip install matplotlib

In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [6]:
data = ("/content/drive/MyDrive/Full Pack IA/Module2: Deep Learning/Data ")

In [7]:
train_path_str = '/content/drive/MyDrive/Full Pack IA/Module2: Deep Learning/Data/train'

In [8]:
val_path_str = '/content/drive/MyDrive/Full Pack IA/Module2: Deep Learning/Data/valid'

In [9]:
test_path_str = '/content/drive/MyDrive/Full Pack IA/Module2: Deep Learning/Data/test'

In [10]:
import os
test_classes = os.listdir(test_path_str)

# **3. Pré-proccessing Data**

In [11]:
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, BatchNormalization, Dropout
#Conv2D: couche de convolution 2D pour extraire des caracteristiques des images
#Dense : couche de cnx
#MaxPooling2D : couche de sous echantillonage qui reduit les dimensions
#Flatten : couche qui aplatit les données en entrées en un vecteur
#BatchNormalization : couche qui normalise les activations de la couche précédente
#Dropout : couche de régularisation pour prévenir le sur apprentissage
from sklearn.utils import shuffle
#fonction pour mélanger les ensembles de données

In [12]:
from keras.utils import to_categorical
#to_categorical : fonction pour convertir des vecteurs de classes en matrices de classes binaires

In [13]:
from sklearn.metrics import confusion_matrix, classification_report
# fonctions pour évaluer les modèles de classification

In [14]:
#def du nombre de classe de sortie
num_classes = 4

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [16]:

from tensorflow.keras.applications.vgg16 import preprocess_input, VGG16
#configuration du générateur d'image pour l'entrainement avec augmentation de données
trainGenerator = ImageDataGenerator(
    preprocessing_function = preprocess_input , #prétraitement des image
    rotation_range = 10,  # rotation aléatoire des images dans une plage de 10 degrés
    width_shift_range = 0.3,  # translation horizontale aléatoire des images dans une plage de 30%
    height_shift_range = 0.3,  # translation verticale aléatoire des images dans une plage de 30%
    shear_range = 0.2,
    zoom_range = 0.1,
    horizontal_flip = True,
    vertical_flip = True,
    dtype = 'float32'
)

In [17]:
valGenerator = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    dtype = 'float32'
)

In [18]:
testGenerator = ImageDataGenerator(
    preprocessing_function = preprocess_input,
    dtype = 'float32'
)

In [19]:
train_data = trainGenerator.flow_from_directory(
    train_path_str,                   # chemin du dossier contenant les images d'entraînement
    target_size = (224, 224),         # taille des images en entrée du modèle
    batch_size = 16,                  # nombre d’images par lot (batch)
    class_mode = 'categorical'        # pour la classification multi-classes
)

Found 613 images belonging to 4 classes.


In [20]:
val_data = valGenerator.flow_from_directory(
    val_path_str,                     # chemin du dossier de validation
    target_size = (224, 224),
    batch_size = 16,
    class_mode = 'categorical'
)

Found 72 images belonging to 4 classes.


In [21]:
test_data = testGenerator.flow_from_directory(
    test_path_str,
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical'
)

Found 315 images belonging to 4 classes.


In [22]:
input_shape = (224, 224, 3)
VGG16_model = VGG16 (
    include_top = False,
    weights = "imagenet" ,
    input_shape = input_shape
)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [23]:
for layer in VGG16_model.layers:
    layer.trainable = False

## **Création de Modèle**

In [25]:
from keras import activations
from tensorflow.keras.models import Sequential # Import Sequential from tensorflow.keras.models

#création d'un modèle séquentiel
model = Sequential()

#Ajout du modèle vgg16 pré-entrainé sans la couche fully connected
model.add(VGG16_model)

# Ajout d'une couche de normalisation par lots pour améliorer la stabilité et la vitesse d'apprentissage
model.add(BatchNormalization())

# Ajout d'une couche de pooling max pour réduire la dimensionnalité des données
model.add(MaxPooling2D(2, 2))

# Aplatissement des données pour les préparer à l'entrée des couches Fully connected
model.add(Flatten())

# Première couche fully connected avec 1024 neurones activés par la fonction ReLU
model.add(Dense(1024, activation='relu'))

# Dropout avec un taux de 30% pour prévenir le surapprentisage
model.add(Dropout(0.3))

# deuxième couche fully connected avec 1024 neurones activés par la fonction ReLU
model.add(Dense(512, activation='relu'))

# Dropout avec un taux de 50% pour prévenir le surapprentisage
model.add(Dropout(0.3))

# troisième couche fully connected avec 1024 neurones activés par la fonction ReLU
model.add(Dense(256, activation='relu'))

#couche fully connected avec 128 neurones activés par la fonction ReLU
model.add(Dense(128, activation='relu'))

#couche de sortie avec un nombre de neurones égal au nombre de classes, activée par softmax pour la classification multi-classes
model.add(Dense(num_classes, activation='softmax'))

#Affichage du résumé du modèle pour vérifier l'architecture et les paramètres
model.summary()



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 7, 7, 512)      │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4608)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │     4,719,616 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,125,892 (76.77 MB)

 Trainable params: 5,410,180 (20.64 MB)

 Non-trainable params: 14,715,712 (56.14 MB)

In [26]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5), #Arrete l'entrainement si la métrique surveillée ne s'ameliore plus aprés 5 epochs
    tf.keras.callbacks.ModelCheckpoint('model.keras', save_best_only=True), #sauvegarde le modèle dans un fichier 'model.keras' #changed the filename to model.keras
    tf.keras.callbacks.TensorBoard(log_dir='logs') #Enregitre les logs pour visualiser les metriques d'entrainement dans TensorBoard
]


In [27]:
# Compile the model
model.compile(optimizer='adam', #choose an optimizer like 'adam' or 'sgd'
              loss='categorical_crossentropy', #use appropriate loss function based on your task (e.g. 'categorical_crossentropy' for multi-class classification)
              metrics=['accuracy']) #specify evaluation metrics

results = model.fit(train_data, validation_data= val_data, epochs=10, verbose=1)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 528s 13s/step - accuracy: 0.3658 - loss: 2.0092 - val_accuracy: 0.5139 - val_loss: 1.4148
Epoch 2/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 423s 10s/step - accuracy: 0.4813 - loss: 1.0734 - val_accuracy: 0.5000 - val_loss: 1.1117
Epoch 3/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 385s 10s/step - accuracy: 0.5361 - loss: 1.1194 - val_accuracy: 0.5000 - val_loss: 0.8988
Epoch 4/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 384s 10s/step - accuracy: 0.5624 - loss: 1.0098 - val_accuracy: 0.4028 - val_loss: 1.1124
Epoch 5/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 381s 10s/step - accuracy: 0.6122 - loss: 0.8859 - val_accuracy: 0.5833 - val_loss: 0.8759
Epoch 6/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 385s 10s/step - accuracy: 0.6215 - loss: 0.8186 - val_accuracy: 0.5556 - val_loss: 0.9276
Epoch 7/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 384s 10s/step - accuracy: 0.6420 - loss: 0.8338 - val_accuracy: 0.6250 - val_loss: 0.9459
Epoch 8/10
39/39 ━━━━━━━━━━━━━━━━━━━━ 385s 10s/step - accuracy: 0.6280 - loss: 0.8033 - val_accuracy: 0.

In [28]:
loss, acc = model.evaluate(test_data, verbose=1)

20/20 ━━━━━━━━━━━━━━━━━━━━ 180s 9s/step - accuracy: 0.6657 - loss: 0.7555


In [29]:
prediction_prob = model.predict(test_data)
prediction = np.argmax(prediction_prob, axis=1)
true_label = test_data.classes
report = classification_report(true_label, prediction)
print(report)

20/20 ━━━━━━━━━━━━━━━━━━━━ 174s 9s/step
              precision    recall  f1-score   support

           0       0.36      0.57      0.44       120
           1       0.00      0.00      0.00        51
           2       0.16      0.15      0.15        54
           3       0.25      0.18      0.21        90

    accuracy                           0.29       315
   macro avg       0.19      0.22      0.20       315
weighted avg       0.24      0.29      0.25       315

